**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install yellowbrick
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

from class_utils.sklearn import (
    make_ext_column_transformer, transformer_extensions
)

import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }

# create a directory for storing any outputs
import os
os.makedirs("data", exist_ok=True)
os.makedirs("output", exist_ok=True)

# create a synthetic dataset
_blobs, _labels = datasets.make_blobs(
    n_samples=600, random_state=3,
    cluster_std=0.75, centers=5
)

_df_blobs = pd.DataFrame(np.hstack([_blobs, _labels.reshape(-1, 1)]),
                         columns=['x', 'y', 'label'])
_df_blobs['y'] *= 100
_df_blobs.to_csv("data/blobs_2d.csv", index=False)

del _blobs
del _labels
del _df_blobs

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
cluster_colors = sns.color_palette()[1:]

def scatter_legend(ax, sc, labels, num_colors, color_array,
                   s, edgecolor):
    handles = []
    
    for i in range(num_colors):
        h = mlines.Line2D([0], [0], ls="", color=color_array[i],
                          ms=s, marker=sc.get_paths()[0],
                          markeredgecolor=edgecolor)
        handles.append(h)

    ax.legend(handles=handles, labels=labels)

def plot_data(
    data, cluster_centres=None, color='b', ax=None,
    cluster_colors=cluster_colors,
    edgecolors='k', labels=None,
    center_color='orange', center_size=200,
    legend=True
):
    if ax is None:
        ax = plt.gca()
        
    if labels is None:
        ax.scatter(data[:, 0], data[:, 1], s=50,
                   color=color, edgecolors=edgecolors)
    else:
        c = np.asarray(cluster_colors)[labels]
        
        sc = ax.scatter(data[:, 0], data[:, 1], s=50,
                        c=c, edgecolors=edgecolors,
                        #cmap=plt.cm.get_cmap('category10', np.max(labels)+1)
                       )
        
        if legend:
            nclusts = np.max(labels)+1
            scatter_legend(ax, sc, ['$c_{}$'.format(i) for i in range(nclusts)],
                           nclusts, cluster_colors, s=6, edgecolor='k')
        
    if not cluster_centres is None:
        ax.scatter(cluster_centres[:, 0],
                   cluster_centres[:, 1],
            s=center_size, c=center_color,
            edgecolors=edgecolors)

    ax.grid(ls='--')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_axisbelow(True)

## Hierarchical Clustering using SciPy

Next we are going to show a practical example of how to apply hierarchical clustering to a simple dataset of 2D point blobs. This will allow us to illustrate the principles and the interfaces, while also making it easy to compare the results of clustering with the 2D plot of the data.

### Loading and Preprocessing the Dataset

First of all, we are going to load the data from a CSV and preprocess it. We are going to be computing distances (Euclidean in our case) and it is therefore going to be very important to standardize our data first.



In [ ]:
# we load the data from the CSV
df = pd.read_csv("data/blobs_2d.csv")

# all inputs are numeric
categorical_inputs = []
numeric_inputs = list(df.columns[:-1])

# the preprocessing pipeline
input_preproc = make_ext_column_transformer(
    (make_pipeline(
        transformer_extensions(
            SimpleImputer(strategy='constant', fill_value='MISSING')
        ),
        OneHotEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        transformer_extensions(
            SimpleImputer()
        ),
        StandardScaler()),
     numeric_inputs),

    return_dataframe=True,
    verbose_feature_names_out=False
)

# the preprocessed data and the classes
df_X = input_preproc.fit_transform(df[categorical_inputs+numeric_inputs])
X = df_X.values
labels = df["label"]

# let's also keep the unnormalized data
X_unnorm = df[categorical_inputs+numeric_inputs].values

# plot the data
plt.figure(figsize=(6, 5))
plot_data(X)

### Computing the Distances and the Linkage

To perform hierarchical clustering, we are going to use the `scipy.cluster.hierarchy` package – `sch` for short. We could also use `AgglomerativeClustering` class from scikit-learn; however, `scipy`'s implementation has better integration with various visualization tools that we are going to be using.

The interface is going to be a bit different from what we are used to in scikit-learn, but the process is still going to be simple enough. Two steps are going to be involved:

# First, we are going to compute the pairwise distances between all our points using `pdist` (with default arguments this computes Euclidean distance, but other options are available).
# Next we compute the linkage using `sch.linkage`, specifying the linkage method. We are going to use the `ward` method, which seeks to minimize the variance of the newly formed clusters.
This "linkage" that we compute is the structure that represents the hierarchical relationships between all our various subclusters.



In [ ]:
D = pdist(X)
L = sch.linkage(D, method='ward')

### Plotting the Dendrogram

We can do various operations with the linkage structure, e.g. plot the visual representation of the hierarchical relationships, which is called a **dendrogram** . We do this by calling `sch.dendrogram`. Given that we have a lot of points, we are also going to specify `no_labels=True` – with so many different labels, they would be illegible in any case.

Once you've plotted the dendrogram and examined it carefully, you will note that the height of the branches in the plot varies. This is because it actually indicates the distance between the two subclusters. You can see that the distance (in terms of the chosen linkage method) between the top level clusters is much larger than at the bottom of the tree.



In [ ]:
G = sch.dendrogram(L, no_labels=True)
plt.xlabel("samples")
plt.ylabel("distance")

plt.savefig("output/hierarchical_scipy_dendrogram.svg", bbox_inches='tight', pad_inches=0)

Note that when doing hierarchical clustering, we do not end up with a flat set of clusters – the output of the clustering algorithm is the linkage structure that we have just visualized. However, getting a flat set of clusters is easy – we just need to pick a threshold at which we are going to cut the dendrogram – that way the points under each branch can be considered a flat cluster. One such cut is already illustrated in our dendrogram – the points under the left branch are coloured orange (cluster 1) and the points under the right branch are coloured green (cluster 2).

Let's try and pick a different threshold, e.g. `5` – we are going to end up with a different set of flat clusters.



In [ ]:
G = sch.dendrogram(L, no_labels=True, color_threshold=5)
plt.xlabel("samples")
plt.ylabel("distance")

plt.savefig("output/hierarchical_scipy_dendrogram_thresh5.svg", bbox_inches='tight', pad_inches=0)

### Computing a Flat Cluster Assignment

To get the assignment of each point from our dataset into a flat cluster, we can use the `sch.fcluster` method, specifying the linkage, the threshold and the criterion (i.e. what we are thresholding). If we want to replicate the flat clustering we visualized above, we specify that we are thresholding on `distance` and again set the threshold to 5.

If you want to know about the other criteria, please refer to `sch.fcluster`'s docstring (e.g. by running `?sch.fcluster` in a cell).

Also, note that `sch.fcluster` has a slight quirk in that the **cluster numbering starts from 1** . Here we deal with that by **subtracting 1**  to get a numbering that starts from 0.



In [ ]:
clusts = sch.fcluster(L, t=5, criterion='distance') - 1

We can now again plot all the original points, colouring them by the computed flat cluster numbers.



In [ ]:
plt.figure(figsize=(6, 5))
plot_data(X, labels=clusts)

### Plotting a Heatmap with Dendrograms

We have already seen that hierarchical clustering allows us to plot a dendrogram, which provides a powerful visualization of the structure of our dataset. A further visualization that can be very useful is a hierarchically clustered heatmap.

The idea is to compute the pairwise distances between all the points in our dataset and visualize them using a heatmap – but also apply hierarchical clustering to both rows and columns of the heatmap so that similar points are grouped together. This gives us even more insight into the spatial structure of our dataset than a dendrogram would.

To make the visualization more robust, we typically compute different clusterings for the rows and for the columns – here we are going to use the `ward` linkage for the one and the `single` linkage for the other.



In [ ]:
L1 = sch.linkage(D, method='ward')
L2 = sch.linkage(D, method='single')

There are different ways to actually plot the heatmap, but perhaps the simplest is to use the `clustermap` function from `seaborn`. We only need to pass the distance matrix and the linkages to it and it takes care of the rest.



In [ ]:
cg = sns.clustermap(squareform(D), row_linkage=L1, col_linkage=L2, rasterized=True)
cg.ax_cbar.set_ylabel("$L_2$ distance")
cg.ax_heatmap.set_xlabel("sample")
cg.ax_heatmap.set_ylabel("sample")

plt.savefig("output/hierarchical_dendrogram_heatmap.svg",
    bbox_inches='tight', pad_inches=0, dpi=300
)

### Plotting a Cluster Map

Note that to get the heatmap above, we have called `clustermap` with a distance matrix and precomputed linkages. When we call `clustermap` with the default arguments on our dataframe, it produces a different kind of visualization called a cluster map.

In this cluster map, rows are clustered as before, but the columns now correspond to the columns of the dataframe. Given this synthetic example, it will probably not be immediately obvious how this kind of visualization is useful. We are therefore going to include a small example on a different dataset to explain it more fully.



In [ ]:
cg = sns.clustermap(df_X)
cg.ax_cbar.set_ylabel("coordinate value")
cg.ax_heatmap.set_xlabel("coordinate")
cg.ax_heatmap.set_ylabel("sample")

### A Cluster Map using Pivoted Data

To facilitate a better understanding of how a cluster map can be useful, we are going to refer to an example presented in [[medium_cluster_map]](#medium_cluster_map), which makes use of the "flights" dataset available through the `seaborn` interface. Let us load it.



In [ ]:
df = sns.load_dataset('flights')
df.head()

As you can see, the dataset has 3 columns. Each row indicates the number of airline passengers in a given month of a given year. Now let's say that we want to look for structure in this data – namely to understand when most of the trips occurred, which months and years are similar to each other, etc. It seems that in order to do that, it would be best to transform our dataset into a matrix format, where each row corresponds to a month, each column corresponds to a year and the entries themselves represent the numbers of passengers.

We can get this kind of representation by pivoting the dataframe:



In [ ]:
df_pivot = df.pivot_table(index="month", columns="year", values="passengers")

#### A Pivoted Cluster Map without Clustering

Now let's plot the cluster map of our pivoted dataframe (at first without any clustering). What we can immediately see is that July and August are the busiest months and also that there is a clear increasing trend in the number of passengers over the observed period.



In [ ]:
cg = sns.clustermap(df_pivot, col_cluster=False, row_cluster=False)
cg.ax_cbar.set_ylabel("passengers")

#### A Pivoted Cluster Map with Clustering

Now let's say that we want to understand about the similarities that exist between the different months and years. All we need to do is to call `clustermap` with clustering on (i.e. no arguments apart from the dataframe).



In [ ]:
cg = sns.clustermap(df_pivot)
cg.ax_cbar.set_ylabel("passengers")

#### A Pivoted Cluster Map with Clustering and Standardization

While the cluster map plotted above gives us an idea about which months and years are similar to each other and what groups there are, we can go even further.

Let's say that what we really want to understand is how the months relate to each other and what groups there exist among them. In that case we do not want the changing yearly totals to obscure these relationships. To make sure that they do not, we can rescale the numbers so that they range from 0 to 1 for each year. This functionality is built in, so all we need to do is specify `standard_scale=1`. Since we now explicitly care about relationships between months and not years, it is probably going to make more sense to turn the clustering off for columns and let the years appear in their natural order: we can do that by passing `col_cluster=False`.

Now the picture is even clearer. It is obvious that overall we get the most flights during the summer holidays – especially in July and August, but also, to a lesser degree in June and September. Over the observed period, people tended to fly much less in the winter months – November, January and February; December was an exception, probably because of the holidays. The remaining months are (together with December) somewhere in the middle.



In [ ]:
cg = sns.clustermap(df_pivot, standard_scale=1, col_cluster=False)
cg.ax_cbar.set_ylabel("passengers")

### References

<a id="medium_cluster_map">[medium_cluster_map]</a> Keith Brooks. Day (4) — Data Visualization — How to use Seaborn for Heatmaps. URL: <https://medium.com/@kbrook10/day-4-data-visualization-how-to-use-seaborn-for-heatmaps-bf8070e3846e>.

